In [1]:
# Set input parameters here
base_args = [
    "-i:email-Eu-core.txt",
    "-d:Y",
    "-m:FFLoop"
]
# Generate n random numbers as seeds
n = 10

In [2]:
exe_paths = [
    "./localmotifclustermain.exe",
    "./purelylocalmotifclustermain.exe"
]

In [3]:
import os
for arg in base_args:
    if arg.startswith("-i:"):
        input_file = arg.split(":")[1]
max_node = 0
if os.path.exists(input_file):
    with open(input_file, 'r') as f:
        for line in f:
            if line.startswith("#"):
                continue
            from_node, to_node = map(int, line.split())
            max_node = max(max_node, from_node, to_node)
    print(f"max node: {max_node}")
else:
    print(f"Error: Input file {input_file} not found.")

max node: 1004


In [4]:
import random
def seed(n, m):
    random_numbers = [random.randint(1, m) for _ in range(n)]
    return random_numbers
seeds = seed(n, max_node)

In [5]:
import platform
def write_system_info(runtime_log_file):
    system_info = {
        "System": platform.system(),
        "Release": platform.release(),
        "Version": platform.version(),
        "Machine": platform.machine(),
        "Processor": platform.processor(),
        "Architecture": platform.architecture()
    }
    with open(runtime_log_file, "w") as runtime_log:
        for key, value in system_info.items():
            runtime_log.write(f"{key}: {value}\n")
        runtime_log.write("\n")

In [6]:
import re
def compare_clusters(log_file1, log_file2):
    """Compare the numbers in the third-to-last line of two log files, identifying which numbers are missing or extra."""
    # Extract numbers from the third-to-last line of a log file
    def extract_numbers_from_third_last_line(log_file):
        with open(log_file, "r") as f:
            lines = f.readlines()
            third_last_line = lines[-3]  # The third-to-last line
            # Use regular expressions to extract numbers from the line
            numbers = re.findall(r'\d+', third_last_line)
            return set(numbers)  # Return the set of numbers

    # Get the numbers from the third-to-last line of both log files
    numbers1 = extract_numbers_from_third_last_line(log_file1)
    numbers2 = extract_numbers_from_third_last_line(log_file2)

    # Identify numbers missing or extra in the second file
    numbers_missing_in_2 = numbers1 - numbers2  # Numbers in the first file but not in the second
    numbers_extra_in_2 = numbers2 - numbers1    # Numbers in the second file but not in the first

    return numbers_missing_in_2, numbers_extra_in_2

In [7]:
# log directory
log_directory = f"./logs/{input_file}"
if not os.path.exists(log_directory):
    os.makedirs(log_directory)

# runtime.log
runtime_file = os.path.join(log_directory, "runtime.log")
write_system_info(runtime_file)
with open(runtime_file, "a") as f:
    f.write(f"Running {exe_paths} with arguments: {' '.join(base_args)}\n\n")

In [8]:
def get_runtime(log_file):
    """Get the last line of the log file"""
    with open(log_file, "r") as f:
        # Move the file pointer to the end of the file
        f.seek(0, os.SEEK_END)
        
        # Get the current position of the file pointer (i.e., the offset at the end of the file)
        position = f.tell()
        
        # Read backwards from the end of the file until a newline character is encountered
        while position > 0:
            position -= 1
            f.seek(position)
            char = f.read(1)
            if char == "\n" and position != f.tell():
                break
        
        # Read the last line
        last_line = f.readline().strip()  # Remove any trailing newline character
        return last_line

In [9]:
# main
import subprocess
for idx in range(0, n):
    current_seed = seeds[idx]
    with open(runtime_file, "a") as f:
        f.write(f"-s:{current_seed} ")

    previous_log_file = None
    # Construct the command for each executable file
    for exe_path in exe_paths:
        args = base_args + [f"-s:{current_seed}"]
        print(f"Running {exe_path} with arguments: {' '.join(args)}")
        # Set the log file name
        log_file = os.path.join(log_directory, f"{os.path.basename(exe_path)}_seed{current_seed}.log")
        
        try:
            # Run the command and capture the output
            result = subprocess.run([exe_path] + args, capture_output=True, text=True)
            # Check if the program ran successfully
            result.check_returncode()  # This will raise an exception if the return code is non-zero
            # Record the success log
            with open(log_file, "w") as f:
                f.write(result.stdout)
            print(f"Success")
        
        except subprocess.CalledProcessError as e:
            # Record the failure log
            with open(log_file, "w") as f:
                f.write(f"Error running {exe_path}\n")
                f.write("Error Output:\n")
                f.write(e.stderr)
            print(f"Error")
            print(f"Error Output:\n{e.stderr}")
        
        # Get and write the last line of the log file to runtime.log
        last_line = get_runtime(log_file)
        with open(runtime_file, "a") as f:
            f.write(last_line + " ")
        
        # If the same seed has been run before
        if previous_log_file:
            missing, extra = compare_clusters(previous_log_file, log_file)
            # Write the data to runtime.log
            with open(runtime_file, "a") as f:
                f.write(f"Missing:{missing} ")
                f.write(f"Extra:{extra} \n")
        
        # Record the current log file to compare next time
        previous_log_file = log_file

Running ./localmotifclustermain.exe with arguments: -i:email-Eu-core.txt -d:Y -m:FFLoop -s:377
Success
Running ./purelylocalmotifclustermain.exe with arguments: -i:email-Eu-core.txt -d:Y -m:FFLoop -s:377
Success
Running ./localmotifclustermain.exe with arguments: -i:email-Eu-core.txt -d:Y -m:FFLoop -s:933
Success
Running ./purelylocalmotifclustermain.exe with arguments: -i:email-Eu-core.txt -d:Y -m:FFLoop -s:933
Success
Running ./localmotifclustermain.exe with arguments: -i:email-Eu-core.txt -d:Y -m:FFLoop -s:553
Success
Running ./purelylocalmotifclustermain.exe with arguments: -i:email-Eu-core.txt -d:Y -m:FFLoop -s:553
Success
Running ./localmotifclustermain.exe with arguments: -i:email-Eu-core.txt -d:Y -m:FFLoop -s:730
Success
Running ./purelylocalmotifclustermain.exe with arguments: -i:email-Eu-core.txt -d:Y -m:FFLoop -s:730
Success
Running ./localmotifclustermain.exe with arguments: -i:email-Eu-core.txt -d:Y -m:FFLoop -s:764
Success
Running ./purelylocalmotifclustermain.exe with ar